# Predicting House Sale Prices

## Introduction

In this project, my second one in Dataquest for Data Scientist path, we'll try to predict the price a house is sold based on some features via Machine Learning algorithms, specifically Linear Regression.

### Dataset

We will work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).



Let's first import some libraries, read the data and setup some functions that will be used as a pipeline to iterate on different models.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,41.0,4920,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,39.0,5389,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,60.0,7500,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,189000


In [3]:
def transform_features(train_df):
    return train_df

In [4]:
def select_features(train_df):
    return train_df[['Gr Liv Area', 'SalePrice']]

In [5]:
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    train_num = train.select_dtypes(include=['integer', 'float'])
    test_num = test.select_dtypes(include=['integer', 'float'])
    
    features = train_num.columns.drop("SalePrice")
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    
    mse = mean_squared_error(test["SalePrice"], predictions)
    return mse**0.5

In [6]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

## Feature Engineering

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns.

We can start by:
- Dropping rows with > 25% missing values
- Dropping text columns with 1+ missing values
- Replace missing values in numerica columns by it's mode (equals to the mean in a normal distribution, more accurate for asymmetrical)

In [7]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    return df

### Enhancements

_What new features can we create, that better capture the information in some of the features?_

Since some columns represent years, it's better to develop a delta indicator to understand that feature better in ML, so we can go ahead and create some year_diff columns


In [8]:
years_sold = df['Yr Sold'] - df['Year Built']
print(years_sold[years_sold < 0])

print('\n')

years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
print(years_since_remod[years_since_remod < 0])

2180   -1
dtype: int64


1702   -1
2180   -2
2181   -1
dtype: int64


The results above show that we can remove those negative values, but the definition might be helpful

Based on the [documentation](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) we can note that other columns can be deprecated as well:
- PID
- Order
- Mo Sold
- Sale Condition
- Sale Type
- Yr Sold

Since they're either not useful for ML or leak info about the final sale.
Now let's update the function and move on!

In [9]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df.drop([1702, 2180, 2181], axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

## Feature Selection


Now we can work with the correlation to accurately choose features.

In [10]:
transform_df = transform_features(df)
df_num = transform_df.select_dtypes(include=['int', 'float'])
corr_coeffs_norm = df_num.corr()['SalePrice'].abs().sort_values()
corr_coeffs_norm

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [11]:
# Choosing correlation coefficient greater than 0.3
corr_coeffs_norm[corr_coeffs_norm > 0.3]
df_num = df_num.drop(corr_coeffs_norm[corr_coeffs_norm < 0.3].index, axis=1)
df_num

,Overall Qual,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,SalePrice,Years Before Sale,Years Since Remod
0,6,112.0,639.0,1080.0,1656,1656,1,7,2,2.0,528.0,210,62,215000,50,50
1,5,0.0,468.0,882.0,896,896,1,5,0,1.0,730.0,140,0,105000,49,49
2,6,108.0,923.0,1329.0,1329,1329,1,6,0,1.0,312.0,393,36,172000,52,52
3,7,0.0,1065.0,2110.0,2110,2110,2,8,2,2.0,522.0,0,0,244000,42,42
4,5,0.0,791.0,928.0,928,1629,2,6,1,2.0,482.0,212,34,189900,13,12
5,6,20.0,602.0,926.0,926,1604,2,7,1,2.0,470.0,360,36,195500,12,12
6,8,0.0,616.0,1338.0,1338,1338,2,6,0,2.0,582.0,0,0,213500,9,9
7,8,0.0,263.0,1280.0,1280,1280,2,5,0,2.0,506.0,0,82,191500,18,18
8,8,0.0,1180.0,1595.0,1616,1616,2,5,1,2.0,608.0,237,152,236500,15,14
9,7,0.0,0.0,994.0,1028,1804,2,7,1,2.0,442.0,140,60,189000,11,11


Based on the documentation, we can assume that these columns are categorical, and could be later _encoded_ into ordinal values

In [12]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [13]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

count_distinct = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
print(count_distinct)

Central Air      2
Street           2
Land Contour     4
Lot Config       5
Bldg Type        5
Heating          6
Roof Style       6
Foundation       6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
MS SubClass     16
Exterior 2nd    17
Neighborhood    28
dtype: int64


Let's define a threshold of 7, we can modify that later if needed

In [14]:
drop_nonuniq_cols = count_distinct[count_distinct > 7].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)

transform_df

,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,...,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Paved Drive_N,Paved Drive_P,Paved Drive_Y
0,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,0,...,0,0,0,0,0,0,1,0,1,0
1,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,0,...,0,0,0,0,0,0,1,0,0,1
2,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,0,...,0,0,0,0,0,0,1,0,0,1
3,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,0,...,0,0,0,0,0,0,1,0,0,1
4,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,701,...,0,0,0,0,0,0,1,0,0,1
5,9978,6,6,20.0,602.0,0.0,324.0,926.0,926,678,...,0,0,0,0,0,0,1,0,0,1
6,4920,8,5,0.0,616.0,0.0,722.0,1338.0,1338,0,...,0,0,0,0,0,0,1,0,0,1
7,5005,8,5,0.0,263.0,0.0,1017.0,1280.0,1280,0,...,0,0,0,0,0,0,1,0,0,1
8,5389,8,5,0.0,1180.0,0.0,415.0,1595.0,1616,0,...,0,0,0,0,0,0,1,0,0,1
9,7500,7,5,0.0,0.0,0.0,994.0,994.0,1028,776,...,0,0,0,0,0,0,1,0,0,1


Seems like this might work, so let's update the function __select_features__!

In [15]:
def select_features(df, coeff_threshold=0.3, uniq_threshold=7):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

## Final touches

Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's create a function and then add a parameter __k__ that controls the type of cross validation that occurs.

In [16]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

Finally, let's see how all this looks like!!

In [17]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[25822.51010627191, 25718.630413284573, 36832.269098552206, 27834.589103176706]


29051.99968032135